In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import time
import pandas as pd

from netunicorn.base import Pipeline, Task, Success, Failure
from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture
from netunicorn.library.tasks.upload.fileio import UploadToFileIO
from netunicorn.library.tasks.upload.webdav import UploadToWebDav
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
from netunicorn.library.tasks.video_watchers.twitch_watcher import WatchTwitchStream

In [3]:
netunicorn_login = 'cs190n12'
netunicorn_password = 'drheuiW4'
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)


In [4]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

Health Check: True
[[snl-server-5], <Uncountable node pool with next node template: [aws-fargate-A-cs190n12-, aws-fargate-B-cs190n12-, aws-fargate-ARM64-cs190n12-]>]


#1- raspi-e4:5f:01:8d:f5:95  
#2- raspi-e4:5f:01:78:6f:2e   
#3- raspi-e4:5f:01:a0:32:5f   
#4- raspi-e4:5f:01:a7:b1:e5   
#5- raspi-e4:5f:01:84:7f:76   
#6- raspi-e4:5f:01:ad:c9:3f   
#7- raspi-e4:5f:01:9c:25:7d   
#8- raspi-e4:5f:01:56:d8:f3   
#9- raspi-e4:5f:01:ac:ed:5b   
#10- raspi-e4:5f:01:ad:c9:04   

In [5]:
working_node = 'snl-server-5'

### Instructions for Uploading PCAPS to lab server
Use the `UploadToWebDav` task from `netunicorn.library.tasks.upload.webdav`  
    - filepaths is a Set of filenames to upload to the endpoint  
    - endpoint is where we will upload the files  
        - http://snl-server-5.cs.ucsb.edu/cs190n/<netunicorn_login\>/<any path you want\>  
        - Corresponding file will be at snl-server-5.cs.ucsb.edu:/mnt/md0/cs190n/<netunicorn_login\>/<path you specified\>/<container_id\>/<filepath\>  
    - username/password: `uploader`/`uploader`  

In [6]:
pipeline = Pipeline()

# Flag to enable early stopping -- so if any task fails pipeline would go on working
# pipeline.early_stopping = False

# Generate data for Vimeo
# Generate data for Twitch

pipeline.then(StartCapture(filepath="/tmp/skanda_youtube_capture1.pcap", name="skanda_youtube_capture1"))
for _ in range(2):
    pipeline.then(WatchYouTubeVideo("https://www.youtube.com/watch?v=Hl24v2Ovs5g", 5))
pipeline.then(StopNamedCapture(start_capture_task_name="skanda_youtube_capture1"))

pipeline.then(SleepTask(2))

# Upload Data
pipeline.then(UploadToWebDav(filepaths={"/tmp/skanda_youtube_capture1.pcap"}, endpoint="http://snl-server-5.cs.ucsb.edu/cs190n/test-cs190n12/", username="uploader", password="uploader"))

Pipeline(2092e419-bb35-4920-a762-c13e6849f2cd): {'root': [<netunicorn.library.tasks.capture.tcpdump.StartCapture object at 0x7f0a71576410>], 1: [<netunicorn.library.tasks.video_watchers.youtube_watcher.WatchYouTubeVideo object at 0x7f0a71575e10>], 2: [<netunicorn.library.tasks.video_watchers.youtube_watcher.WatchYouTubeVideo object at 0x7f0a715761a0>], 3: [<netunicorn.library.tasks.capture.tcpdump.StopNamedCapture object at 0x7f0a715761d0>], 4: [<netunicorn.library.tasks.basic.SleepTask with name 87fb88e7-2cc9-4ed5-a401-550511b842ea>], 5: [<netunicorn.library.tasks.upload.webdav.UploadToWebDav object at 0x7f0a71576e60>]}

In [7]:
working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)

# Creating the experiment
experiment = Experiment().map(pipeline, working_nodes)
print(experiment)

 - Deployment: Node=snl-server-5, executor_id=, prepared=False, error=None


### Installing Libraries vs Docker Image

In [8]:
for line in experiment[0].environment_definition.commands:
    print(line)

sudo apt-get update
sudo apt-get install -y tcpdump
apt install -y python3-pip wget xvfb procps chromium chromium-driver
pip3 install selenium webdriver-manager
apt install -y python3-pip wget xvfb procps chromium chromium-driver
pip3 install selenium webdriver-manager
sudo apt-get update
sudo apt-get install -y tcpdump
sudo apt-get install -y procps
sudo apt-get install -y curl


We can also use a predefined Docker image which will avoid installing any libraries.

In [9]:
from netunicorn.base import DockerImage
for deployment in experiment:
    # you can explore the image on the DockerHub
    deployment.environment_definition = DockerImage(image='satyandraguthula/netunicorn_images')

In [10]:
experiment_label = "da1aco113c1i0ns"

Now we can prepare the experiment, check for any errors and execute.

In [11]:
try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY


In [12]:
for deployment in client.get_experiment_status(experiment_label).experiment:
    print(f"Prepared: {deployment.prepared}, error: {deployment.error}")

Prepared: True, error: None


In [13]:
client.start_execution(experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED


In [14]:
from returns.pipeline import is_successful

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    result, log = report.result  # report stores results of execution and corresponding log
    
    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    data = result.unwrap() if is_successful(result) else result.failure()
    for key, value in data.items():
        print(f"{key}: {value}")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

Node name: snl-server-5
Error: None
Result is: <class 'returns.result.Success'>
skanda_youtube_capture1: [<Success: 8>]
23da1fa5-cb40-473e-a36b-3512a1ee98dd: [<Success: Video finished by timeout: 5 seconds>]
6bfd6190-411e-450d-81af-6128f0f1ed06: [<Success: Video finished by timeout: 5 seconds>]
311009e5-f026-4a4e-bbb9-b8f6d51bb960: [<Success: >]
87fb88e7-2cc9-4ed5-a401-550511b842ea: [<Success: 2>]
a971f367-63aa-4800-86b4-8e80566a5733: [<Success: [<Success:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.4M    0     0  100 11.4M      0   164M --:--:-- --:--:-- --:--:--  162M
>]>]
Parsed configuration: Gateway located on https://pinot.cs.ucsb.edu/netunicorn/gateway
Current directory: /
Successfully received the execution graph.
Execution is finished, start reporting results.



In [29]:
from util.pandas_util import split_pcap_to_csv


In [30]:
# df = pcap_to_df('/mnt/md0/cs190n/test-cs190n12/b3defa55-cffb-44ec-9dd3-0071693da945/tmp/skanda_youtube_capture1.pcap')
split_pcap_to_csv('/mnt/md0/cs190n/test-cs190n12/b3defa55-cffb-44ec-9dd3-0071693da945/tmp/', '/mnt/md0/cs190n/test-cs190n12/b3defa55-cffb-44ec-9dd3-0071693da945/tmp/skanda_youtube_capture_test.csv')

PermissionError: [Errno 13] Permission denied: '/mnt/md0/cs190n/test-cs190n12/b3defa55-cffb-44ec-9dd3-0071693da945/tmp/skanda_youtube_capture_test.csv'

In [31]:
df = pd.read_csv('skanda_yt_capture_1.csv')
df.head()

/tmp/ipykernel_2502894/2173566017.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('skanda_yt_capture_1.csv')


,Unnamed: 0,version,ihl,tos,len,id,flags,frag,ttl,proto,...,tcp_reserved,tcp_flags,tcp_window,tcp_chksum,tcp_urgptr,tcp_options,udp_sport,udp_dport,udp_len,udp_chksum
0,0,4,5,0,58,59943,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11707.0
1,1,4,5,0,58,57513,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11708.0
2,2,4,5,0,58,58646,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,48218.0
3,3,4,5,0,58,59944,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11707.0
4,4,4,5,0,58,57514,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11708.0


In [35]:
df

,Unnamed: 0,version,ihl,tos,len,id,flags,frag,ttl,proto,...,tcp_reserved,tcp_flags,tcp_window,tcp_chksum,tcp_urgptr,tcp_options,udp_sport,udp_dport,udp_len,udp_chksum
0,0,4,5,0,58,59943,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11707.0
1,1,4,5,0,58,57513,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11708.0
2,2,4,5,0,58,58646,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,48218.0
3,3,4,5,0,58,59944,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11707.0
4,4,4,5,0,58,57514,DF,0,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,55557.0,53.0,38.0,11708.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84318,84318,4,5,0,52,39986,NaN,0,58,6,...,0.0,FA,1043.0,44029.0,0.0,3.0,NaN,NaN,NaN,NaN
84319,84319,4,5,0,52,30891,DF,0,64,6,...,0.0,A,501.0,47173.0,0.0,3.0,NaN,NaN,NaN,NaN
84320,84320,4,5,0,52,61229,NaN,0,58,6,...,0.0,FA,1044.0,23452.0,0.0,3.0,NaN,NaN,NaN,NaN
84321,84321,4,5,0,52,26688,DF,0,64,6,...,0.0,A,501.0,15753.0,0.0,3.0,NaN,NaN,NaN,NaN
